## Telemetry Logging with AutoGen 

AutoGen offers some utilities to log telemetry data, useful for debugging and performance analysis. This notebook demonstrates how to use these utilities. 

In general, users can initiate logging by calling `autogen.telemetry.start_logging` and stop logging by calling `autogen.telemetry.stop_logging`, and get logged data by calling `autogen.telemetry.get_log`.

In [1]:
import os
import json
import autogen
from autogen import AssistantAgent, UserProxyAgent
import pandas as pd  ## install pandas if not installed

# start logging telemetry data
telemetry_session_id = autogen.telemetry.start_logging(dbpath="telemetry.db")
print("Telemetry session ID: " + str(telemetry_session_id))

## Create an Agent Workflow and Run it
llm_config = {
    "config_list": [
        {
            "model": "gpt-4-turbo-preview",
            "api_key": os.environ["OPENAI_API_KEY"],  # Note, add your own API key here
        }
    ],
    "temperature": 0.9,
}

## start_logging
assistant = AssistantAgent(name="assistant", llm_config=llm_config)
user_proxy = UserProxyAgent(
    name="user_proxy",
    code_execution_config=False,
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: "TERMINATE" in msg["content"],
)

user_proxy.initiate_chat(
    assistant, message="What is the height of the Eiffel Tower? Only respond with the answer and terminate"
)
autogen.telemetry.stop_logging()

Telemetry session ID: 2502ff49-ffbf-4ab3-888b-77dbba663f44
user_proxy (to assistant):

What is the height of the Eiffel Tower? Only respond with the answer and terminate

--------------------------------------------------------------------------------
assistant (to user_proxy):

The height of the Eiffel Tower is 330 meters (1,083 feet).

TERMINATE

--------------------------------------------------------------------------------


## Getting Data from the SQL Lite Database 

We can get data from the database by calling `autogen.telemetry.get_log`. This function returns a pandas dataframe with the logged data. 



In [2]:
def str_to_dict(s):
    return json.loads(s)


telemetry_data = autogen.telemetry.get_log(table="chat_completions")
telemetry_df = pd.DataFrame(telemetry_data)
telemetry_df["total_tokens"] = telemetry_df.apply(
    lambda row: str_to_dict(row["response"])["usage"]["total_tokens"], axis=1
)
telemetry_df

,id,invocation_id,client_id,wrapper_id,session_id,request,response,is_cached,cost,start_time,end_time,total_tokens
0,1,962a7e9d-e0d0-48e9-a0f1-bfa380c75724,140127801634544,140127742324752,2dae4d47-79f4-4fbb-8ffc-69c2e6213799,"{""messages"": [{""content"": ""You are a helpful A...","{\n ""id"": ""chatcmpl-8nBekDzT58BHYVz8KeVcgtv...",1,0.0,2024-01-31 21:31:35.475838,2024-01-31 21:31:35.475989,511
1,2,ae038fc5-00de-48e3-b18f-f54bc6b9bdea,140292698273392,140292698272000,8695ecc6-eac4-4a1e-b147-d317416e64ed,"{""messages"": [{""content"": ""You are a helpful A...","{\n ""id"": ""chatcmpl-8nBekDzT58BHYVz8KeVcgtv...",1,0.0,2024-01-31 21:37:40.485350,2024-01-31 21:37:40.485631,511
2,3,6b849f31-95b5-47f1-b1cf-49a3bd9746f9,140292724011792,140292724245216,22fe9e28-c889-49b2-a100-89b61d93d092,"{""messages"": [{""content"": ""You are a helpful A...","{\n ""id"": ""chatcmpl-8nBekDzT58BHYVz8KeVcgtv...",1,0.0,2024-01-31 21:37:46.964779,2024-01-31 21:37:46.964931,511
3,4,bb06d6e4-d1d8-43a0-9ab4-ce56037dd0f1,139864741475808,139864741476048,2502ff49-ffbf-4ab3-888b-77dbba663f44,"{""messages"": [{""content"": ""You are a helpful A...","{\n ""id"": ""chatcmpl-8nBekDzT58BHYVz8KeVcgtv...",1,0.0,2024-01-31 21:37:54.907778,2024-01-31 21:37:54.907935,511


## Computing Cost 

One use case of telemetry data is to compute the cost of a session.

In [4]:
# total tokens for all sessions .. sum totoal tokens for all sessions
total_tokens = telemetry_df["total_tokens"].sum()
# total tokens for specific session telemetry_session_id
session_tokens = telemetry_df[telemetry_df["session_id"] == telemetry_session_id]["total_tokens"].sum()

print("Total tokens for all sessions: " + str(total_tokens))
print("Total tokens for session " + str(telemetry_session_id) + ": " + str(session_tokens))

Total tokens for all sessions: 2044
Total tokens for session 2502ff49-ffbf-4ab3-888b-77dbba663f44: 511
